In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList
import torch
import re

# Check for GPU and set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Set eos_token as pad_token for the tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load the model, set pad_token_id and move to device
model = AutoModelForCausalLM.from_pretrained(
	"gpt2", 
	pad_token_id=tokenizer.eos_token_id,
	torch_dtype=torch.float16,
	device_map="auto"
)

In [2]:
print(hasattr(model, "device"))

True


In [3]:
def prepare_inputs(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    if not hasattr(model, "device"):
        return inputs
    return {k: v.to(model.device) for k, v in inputs.items()}

In [4]:
def get_response(prompt, tokenizer, model):
    inputs = prepare_inputs(prompt, tokenizer, model)
    output = model.generate(
        **inputs,
        max_length=50,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id 
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [5]:
res = get_response("The future of AI is", tokenizer, model)

print(res)

The future of AI is in the hands of the next generation of researchers.

"We are in a very exciting time in AI," said Dr. Michael S. Hirsch, a professor of computer science at the University of California, Berkeley.


In [6]:
def get_response_sample(prompt, tokenizer, model):
    inputs = prepare_inputs(prompt, tokenizer, model)
    output = model.generate(
        **inputs,
        max_length=50,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id   
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [7]:
res = get_response_sample("The future of AI is", tokenizer, model)

print(res)

The future of AI is clear. A computer can be programmed to be able to learn and adapt to the world, and it can learn from a computer that can adapt itself.

This is where AI comes in. The future will be much different


In [8]:
def get_response_improved(prompt, tokenizer, model):
    inputs = prepare_inputs(prompt, tokenizer, model)
    output = model.generate(
        **inputs,
        max_length=50,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id, 
        eos_token_id=tokenizer.eos_token_id   
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [9]:
res = get_response_improved("The future of AI is", tokenizer, model)

print(res)

The future of AI is uncertain. But we're still seeing the emergence and development of tools that can be used to manage, analyze, predict, control information in real-time."
: "In this paper, I'll show you how our algorithm


In [10]:
class StopOnlyAtPunctuation(StoppingCriteria):
    def __init__(self, stop_words:list):
        self.stop_words = stop_words

    def __call__(self, input_ids, scores, **kwargs):
        # Ensure clean text
        decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)

        suffixes = tuple(self.stop_words)

        if decoded_text.endswith(suffixes):
            return True
        return False


stopping_criteria = StoppingCriteriaList([StopOnlyAtPunctuation(stop_words=[".", "!"])])

In [11]:
def get_response_with_stop(prompt, tokenizer, model):
    inputs = prepare_inputs(prompt, tokenizer, model)
    output = model.generate(
        **inputs,
        max_length=50,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id, 
        eos_token_id=tokenizer.eos_token_id,
        stopping_criteria=stopping_criteria  # Add custom stopping criteria   
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [12]:
res = get_response_with_stop("The future of AI is", tokenizer, model)

print(res)

The future of AI is not yet clear, but some research suggests it may soon be possible to do so.


In [13]:
class StopAtPunctuation(StoppingCriteria):
    def __init__(self, stop_words, min_sentences=2):
        self.stop_words = stop_words
        self.min_sentences = min_sentences

    def __call__(self, input_ids, scores, **kwargs):
        # Ensure clean text
        decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)

        suffixes = tuple(self.stop_words)

        # Convert list into a regex pattern
        regex = f"[{''.join(re.escape(word) for word in self.stop_words)}]+"
        matches = re.findall(regex, decoded_text)

        # Count the number of complete sentences
        sentence_count = len(matches)

        if sentence_count >= self.min_sentences and decoded_text.endswith(suffixes):
            return True
        
        return False
    
# Require at least 3 sentences before stopping
stopping_criteria = StoppingCriteriaList([StopAtPunctuation(stop_words=[".", "!"], min_sentences=2)])

In [14]:
def get_response_final(prompt, tokenizer, model):
    inputs = prepare_inputs(prompt, tokenizer, model)
    output = model.generate(
        **inputs,
        max_length=100,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id, 
        eos_token_id=tokenizer.eos_token_id,
        stopping_criteria=stopping_criteria  # Add custom stopping criteria   
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [15]:
res = get_response_final("The future of AI", tokenizer, model)

print(res)

The future of AI is uncertain. We may well be able to develop a machine that can help us understand and control many things, but we have yet not done it before."


 "This has been my first experience with the technology," said Professor Lawrence Krauss in an interview from Zurich on Thursday (11 July).
